In [ ]:
# --- INSTALL LIBRARY YANG BELUM ADA ---
!pip install Sastrawi
!pip install googletrans==4.0.0-rc1
!pip install vaderSentiment
import os

# --- IMPORT LIBRARY ---
from googleapiclient.discovery import build
import pandas as pd
import getpass
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.display import FileLink
from google.colab import files
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [ ]:
# --- AUTH YOUTUBE API ---
api_key = getpass.getpass('masukin api code lu: ')
playlist_ids = ['PLLunBNCsgciIkoNTQnp3XRhM8lqqd8FQm']
youtube = build('youtube', 'v3', developerKey=api_key)

masukin api code lu: ··········


In [ ]:
# --- STEP 1: Get Video IDs from Playlist ---
def get_all_video_ids_from_playlists(youtube, playlist_ids):
    all_videos = []
    for playlist_id in playlist_ids:
        next_page_token = None
        while True:
            playlist_request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token)
            playlist_response = playlist_request.execute()
            all_videos += [item['contentDetails']['videoId'] for item in playlist_response['items']]
            next_page_token = playlist_response.get('nextPageToken')
            if next_page_token is None:
                break
    return all_videos

video_ids = get_all_video_ids_from_playlists(youtube, playlist_ids)


In [ ]:
# --- STEP 2: Get Comments from Videos ---
def get_replies(youtube, parent_id, video_id):
    replies = []
    next_page_token = None
    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()
        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment.get('updatedAt', comment['publishedAt'])
            })
        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break
    return replies

def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None
    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()
        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': top_comment['textDisplay'],
                'Date': top_comment.get('updatedAt', top_comment['publishedAt'])
            })
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))
        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break
    return all_comments

all_comments = []
for video_id in video_ids:
    all_comments.extend(get_comments_for_video(youtube, video_id))

comments_df = pd.DataFrame(all_comments)
comments_df.to_csv('comments_data.csv', index=False)
files.download('comments_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>